### 🏥 Hospital Database Agent - Main Execution Notebook

This notebook serves as the **main execution environment** for an intelligent agent designed to answer natural language questions about a hospital database.

---

#### 📌 Purpose

This notebook demonstrates the functionality, integration, and usage of an AI-powered agent capable of interpreting user queries and retrieving accurate responses from a structured hospital database.

---

#### ⚙️ Scope and Capabilities

The agent is currently capable of handling:

- ✅ **Basic queries** (e.g., patient names, doctor specialties, medicine stock levels etc.)
- ✅ **Intermediate queries** (e.g., rooms cleaned today, patients prescribed a capsule etc)
- ✅ **Some complex queries** (e.g., avg. time to complete an ambulance ride, patients with total billed amount for a certain period etc.)

**Limitations**:
- ❌ May struggle with highly nested or ambiguous queries
- ❌ Limited understanding of external clinical knowledge or nuanced medical context. For instance, it does not know what ICD-10 codes mean even though they are present in the database.

---

#### 🧱 Architecture Overview

The implementation is modular, with logic split into different files for clarity and maintainability:

- `lang_graph.py` – Lang Graph defined here. The edges, nodes, control flow for graph etc.
- `tools.py` – Agent tools for SQL parsing, schema understanding, error handling etc.
- `utils.py` – General-purpose utility functions along with the function used to run tests
- `agent.py` – The main agent is defined here. Also the critic agent definition is in this file.
-  `SQL_Files\` -  Contains the SQL scripts used to populate the database.

---

#### ▶️ How to Use This Notebook

1. Run all initialization cells to load the agent and dependencies
2. Populate the database using the SQL scripts provided in the `SQL_Files\` folder.
3. Provide the OPEN AI API key and TAVILY API key in the input cell.
4. Run the main execution cell to start the agent.
5. Use the input cell to ask natural language questions (e.g., *"How many patients were admitted in July?"*)
6. Review the generated SQL and the returned results

---

#### 🧪 Summary

This notebook provides a flexible, extensible platform for interacting with hospital data using natural language. It showcases the power of combining NLP with structured data systems, and acts as a foundation for building more intelligent and clinically aware data agents.



In [2]:
import getpass
import os
import sqlite3
import pandas as pd
from utils import execute_sql_script, evaluate
from langchain_openai.chat_models import ChatOpenAI
from agent import LangChain

### 🗂️ Database Setup Instructions

The following code is used to **create and initialize the hospital database**.

Please ensure that all SQL files are placed inside a directory named **`SQL_Files`**, which should be located in the **same root folder as this notebook**.

---

#### 📄 Contents of the Database

The database has been designed to mimic a realistic hospital environment using **dummy data**. It contains multiple interrelated tables, including:

- 🧑‍⚕️ `Patients`
- 👨‍⚕️ `Doctors`
- 🧾 `Billing`
- 📅 `Appointments`
- 🏨 `Rooms`
- … and more

Several domain-relevant fields have been included to simulate real-world hospital data scenarios, such as:

- ✅ **ICD-10 codes** (for diagnoses)
- ✅ **CMS Risk Score** (for risk stratification)

---

💡 *Tip: Make sure your SQL scripts are run in the correct order to avoid dependency issues between tables (e.g., foreign key constraints).*


In [3]:
conn = sqlite3.connect("my_database.db")
cursor = conn.cursor()
execute_sql_script(cursor, "Drop_All.sql")
execute_sql_script(cursor, "Table_Creation.sql")
execute_sql_script(cursor, "Hospital_Data.sql")
conn.commit()
conn.close()


### 🔐 API Keys Required

This project requires access to two external APIs to enable intelligent query processing and optional web-assisted context retrieval:

---

#### 🧠 OpenAI API

The OpenAI API provides the core **language model** capabilities used by the agent to:

- Interpret natural language questions
- Generate relevant SQL queries
- Understand database schema and relationships

- [Sign up for OpenAI API Key](https://platform.openai.com/signup)
- [OpenAI API Keys Dashboard](https://platform.openai.com/account/api-keys)

---

#### 🌐 Tavily API

The Tavily API is optionally used to provide **external context** or **web search results** that can assist the agent in answering more complex or knowledge-based queries that go beyond the database.

- [Sign up for Tavily API Key](https://app.tavily.com/sign-up)
- [Tavily API Dashboard](https://app.tavily.com/dashboard)



In [4]:
OPENAI_API_KEY = getpass.getpass("Enter OPENAI_API_KEY:")
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [5]:
TAVILY_API_KEY = getpass.getpass("Enter TAVILY_API_KEY:")
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [6]:
model = ChatOpenAI(model='gpt-4o', openai_api_key=os.getenv("OPENAI_API_KEY"))
DB_PATH = os.path.abspath("my_database.db")
SQLALCHEMY_URL = f"sqlite:///{DB_PATH}"
os.environ["SQLITE_URL"] = SQLALCHEMY_URL

## 🚀 Agent Initialization and Execution

In the next two cells, we:

1. **Create an instance** of the main class responsible for executing the agent.
2. **Pass the necessary parameters**, such as model and SQLite URL
3. **Trigger the agent** by calling an execution method, which enables it to start processing user queries.

This setup ensures that the agent is properly initialized and ready to interpret natural language questions, convert them into SQL queries, and return results from the hospital database.


In [7]:
sql_helper = LangChain(model, SQLALCHEMY_URL)

In [8]:
def fetch_and_execute_sql_from_string(question: str):
    sql_query, metadata = sql_helper.get_sql(question)
    if sql_query is not None:
        return sql_helper.run_query(sql_query)
    else:
        print(metadata)
        return "No SQL query found."

### Examples

##### ✅ Below is a fairly simple SQL query where the agent successfully fetches the correct output:

In [9]:
question = "List the names of doctors with their specialties"
fetch_and_execute_sql_from_string(question)

{"sql": "SELECT first_name || ' ' || last_name AS doctor_name, specialty FROM Doctors;", "scratchpad": "To list the names of doctors with their specialties, we need to use the 'Doctors' table as it contains both name-related columns (first_name, last_name) and the 'specialty' column. The required output is a concatenation of first_name and last_name for the doctor's full name along with their specialty.", "tables_used": "Doctors"}


[('Dr. Alice Miller', 'Cardiology'),
 ('Dr. Bob Williams', 'Neurology'),
 ('Dr. Charlie Brown', 'Orthopedics'),
 ('Dr. Diana Jones', 'Dermatology'),
 ('Dr. Eva Garcia', 'Pediatrics'),
 ('Dr. Frank Martinez', 'General Surgery'),
 ('Dr. Grace Rodriguez', 'Gynecology'),
 ('Dr. Henry Lee', 'Psychiatry'),
 ('Dr. Ivy Lopez', 'Ophthalmology'),
 ('Dr. Jack White', 'ENT')]

##### ✅ The agent is asked a relatively more complex question compared to the previous one, and it succeeds.


In [12]:
question = "What is the average time between pickup and dropoff for completed ambulance rides?"
fetch_and_execute_sql_from_string(question)

Calling validate_sql
The SQL query was successfully validated and correctly retrieves the average time between pickup and dropoff for completed ambulance rides. Here's the structured response according to the given schema:

```json
{
    "sql": "SELECT AVG((strftime('%s', dropoff_time) - strftime('%s', pickup_time)) / 60.0) AS average_time_in_minutes\nFROM Ambulance_Log\nWHERE LOWER(status) = LOWER('Completed');",
    "scratchpad": "The `Ambulance_Log` table contains the relevant data. To compute the average time between pickup and dropoff:\n1. Filter entries with `status` 'Completed'.\n2. Use `strftime()` to compute time difference between `dropoff_time` and `pickup_time` in seconds.\n3. Divide by 60 to convert to minutes.\n4. Calculate the average of these differences.",
    "tables_used": "Ambulance_Log"
}
```


[(32.5,)]

##### ✅ The agent is asked to update the values in the billing table. It recognizes that it is not permitted to perform this action and returns a suitable response


In [13]:
question = "Change the billing value to 1000 for all patients"
fetch_and_execute_sql_from_string(question)

The task requires changing the billing value to 1000 for all patients, which implies an update operation on the 'Billing' table where the billing values are stored. However, I can only generate select queries. Thus, instead of performing an update, I return a message indicating that only select queries are allowed.

Here's the structured JSON response with that message:

```json
{
    "sql": "SELECT 'only select query allowed';",
    "scratchpad": "The operation requires updating the 'total_amount' field in the 'Billing' table to 1000 for all rows. As per the constraint, I can only generate select queries, so I will output a statement indicating that only select queries are permitted.",
    "tables_used": ""
}
```
[Critic Feedback]: The SQL you generated does not fully answer the question. Feedback: The SQL provided does not answer the question as it performs a SELECT query that returns a string instead of altering any values. Furthermore, the task itself requires altering data, which 

[('Operation not performed: Only SELECT queries are allowed',)]

##### ⚠️ In the query below, the agent correctly identifies the relevant column but makes an assumption about the "high risk" value. This assumption is noted in the output scratchpad. Issues like this could potentially be avoided by providing explicit column explanations or metadata.


In [14]:
question = "List the patients who are at high risk"
fetch_and_execute_sql_from_string(question)

Calling validate_sql
The SQL query has been validated and is correct. Here's the finalized output:

```json
{
    "sql": "SELECT patient_id, first_name, last_name, risk_score\nFROM Patients\nWHERE risk_score > 7;",
    "scratchpad": "1. The requirement is to find patients at high risk, which can be identified by a score in the 'risk_score' column in the 'Patients' table.\n2. Typical assumption for high-risk thresholds is using risk_score > 7.\n3. The SQL query selects necessary columns from the 'Patients' table where the 'risk_score' is greater than 7.",
    "tables_used": "Patients"
}
```
The query selects patient IDs, first names, last names, and their risk scores, focusing on those with a risk score greater than a threshold of 7, interpreted here as "high risk."


[]

##### ⚠️ In this case, the agent identifies and develops a logic that can be considered valid. However, the more straightforward approach would be to check the room assignment in the `Rooms` table. Although the agent recognizes this, it bypasses it in favor of a different logic. Providing column explanations could help guide the agent toward the simpler solution.


In [15]:
question = "Get the list of all room numbers that are currently occupied."
fetch_and_execute_sql_from_string(question)

Calling validate_sql
Here's the final output with the constructed SQL query:

```json
{
    "sql": "SELECT r.room_number \nFROM Rooms r\nJOIN Room_Assignments ra ON r.room_id = ra.room_id\nWHERE ra.end_date IS NULL;",
    "scratchpad": "The task is to find all room numbers that are currently occupied. In the schema, the 'Room_Assignments' table helps us determine if a room is occupied by checking if 'end_date' is NULL. The 'Rooms' table provides room numbers. Therefore, we join 'Rooms' and 'Room_Assignments' on 'room_id' and filter where 'end_date' is NULL to get the room numbers of currently occupied rooms.",
    "tables_used": "Rooms, Room_Assignments"
}
```


[('ICU101',), ('OPR401',), ('STF501',)]

### Test Cases

#### 🧪 Query Evaluation Logic

The generated SQL query is first compared with the expected query using an **exact match** check.

- ✅ If the queries match exactly, they are considered correct.
- ❌ If the exact match fails, both queries are passed to a **LLM-based judge** to evaluate their **semantic equivalence**.

The LLM judge determines whether the two queries would return **similar or equivalent results**, even if their syntax differs.

- **Equivalent** → The LLM judge confirms that the queries are semantically the same.
- **Not Equivalent** → The LLM judge indicates that the queries differ in meaning or output.

> ⚠️ *Note: The LLM judge is only invoked when the exact match check fails.*

---

#### 🧪 Test Case Execution

Currently, output for **5 test cases** is displayed by default.

If you wish to evaluate more, you can **uncomment the additional test cases** in the code and re-run the notebook.


In [16]:
test_cases = [
  {
    "question": "List the names of patients in the system?",
    "actual_query": "SELECT first_name, last_name FROM Patients;"
  },
  {
    "question": "List the names of doctors with their specialties.",
    "actual_query": "SELECT first_name, last_name, specialty FROM Doctors;"
  },
  {
    "question": "Show all available ambulance numbers.",
    "actual_query": "SELECT ambulance_number FROM Ambulance WHERE availability = 'Available';"
  },
  {
    "question": "Which medicines are currently out of stock?",
    "actual_query": "SELECT name FROM Medicine WHERE stock_quantity = 0;"
  },
  {
    "question": "Get the list of all room numbers that are currently occupied.",
    "actual_query": "SELECT room_number FROM Rooms WHERE status = 'Occupied';"
  },
  # {
  #   "question": "Which doctors work in the Cardiology department?",
  #   "actual_query": "SELECT d.first_name, d.last_name FROM Doctors d JOIN Doctor_Department dd ON d.doctor_id = dd.doctor_id JOIN Departments dept ON dd.department_id = dept.department_id WHERE dept.department_name = 'Cardiology';"
  # },
  # {
  #   "question": "Find patients who have appointments scheduled for tomorrow.",
  #   "actual_query": "SELECT p.first_name, p.last_name, a.appointment_date FROM Appointments a JOIN Patients p ON a.patient_id = p.patient_id WHERE a.appointment_date = DATE('now', '+1 day');"
  # },
  # {
  #   "question": "List the number of staff members in each department.",
  #   "actual_query": "SELECT dept.department_name, COUNT(s.staff_id) AS staff_count FROM Staff s LEFT JOIN Departments dept ON s.department_id = dept.department_id GROUP BY dept.department_name;"
  # },
  # {
  #   "question": "Which rooms were cleaned today?",
  #   "actual_query": "SELECT r.room_number FROM Cleaning_Service cs JOIN Rooms r ON cs.room_id = r.room_id WHERE cs.service_date = DATE('now');"
  # },
  # {
  #   "question": "List all patients who received a capsule medicine.",
  #   "actual_query": "SELECT DISTINCT p.first_name, p.last_name FROM Pharmacy ph JOIN Medicine m ON ph.medicine_id = m.medicine_id JOIN Patients p ON ph.patient_id = p.patient_id WHERE m.type = 'Capsule';"
  # },
  # {
  #   "question": "Which doctor has attended the most unique patients?",
  #   "actual_query": "SELECT d.first_name, d.last_name, COUNT(DISTINCT app.patient_id) AS unique_patients FROM Appointments app JOIN Doctors d ON app.doctor_id = d.doctor_id GROUP BY d.doctor_id ORDER BY unique_patients DESC LIMIT 1;"
  # },
  # {
  #   "question": "Find the total billed amount for each patient along with their names.",
  #   "actual_query": "SELECT p.first_name, p.last_name, SUM(b.total_amount) AS total_billed FROM Billing b JOIN Patients p ON b.patient_id = p.patient_id GROUP BY b.patient_id;"
  # },
  # {
  #   "question": "List all medicines that are prescribed but not available in stock.",
  #   "actual_query": "SELECT DISTINCT m.name, m.brand FROM Medical_Records_Medicine mrm JOIN Medicine m ON mrm.medicine_id = m.medicine_id WHERE m.stock_quantity = 0;"
  # },
  # {
  #   "question": "Which patient has the highest number of ambulance rides?",
  #   "actual_query": "SELECT p.first_name, p.last_name, COUNT(al.log_id) AS ride_count FROM Ambulance_Log al JOIN Patients p ON al.patient_id = p.patient_id GROUP BY p.patient_id ORDER BY ride_count DESC LIMIT 1;"
  # },
  # {
  #   "question": "What is the average time between pickup and dropoff for completed ambulance rides?",
  #   "actual_query": "SELECT AVG((JULIANDAY(dropoff_time) - JULIANDAY(pickup_time)) * 24 * 60) AS avg_duration_minutes FROM Ambulance_Log WHERE status = 'Completed';"
  # }
]

In [17]:
evaluation_results = evaluate(sql_helper, test_cases)
df_results = pd.DataFrame(evaluation_results)

List the names of patients in the system?
{"sql": "SELECT first_name, last_name FROM Patients;", "scratchpad": "The task requires listing the names of patients in the system. The 'Patients' table contains the columns 'first_name' and 'last_name', which are required to list the full names of the patients. Hence, the SQL query was crafted to select these columns from the 'Patients' table.", "tables_used": "Patients"}
List the names of doctors with their specialties.
```json
{
    "sql": "SELECT first_name || ' ' || last_name AS doctor_name, specialty FROM Doctors;",
    "scratchpad": "To find the names of doctors with their specialties, I referred to the 'Doctors' table in the schema. The relevant columns for this query are 'first_name', 'last_name', and 'specialty'. We concatenate 'first_name' and 'last_name' using SQL string concatenation to get the full name of the doctor. This query will list each doctor's full name along with their specialty.",
    "tables_used": "Doctors"
}
```
Sho

In [18]:
df_results

,question,expected_sql,generated_sql,exact_match,llm_judged_equivalent
0,List the names of patients in the system?,"SELECT first_name, last_name FROM Patients;","SELECT first_name, last_name FROM Patients;",True,None
1,List the names of doctors with their specialties.,"SELECT first_name, last_name, specialty FROM D...",SELECT first_name || ' ' || last_name AS docto...,False,Equivalent
2,Show all available ambulance numbers.,SELECT ambulance_number FROM Ambulance WHERE a...,SELECT ambulance_number FROM Ambulance WHERE L...,False,Equivalent
3,Which medicines are currently out of stock?,SELECT name FROM Medicine WHERE stock_quantity...,SELECT name FROM Medicine WHERE stock_quantity...,True,None
4,Get the list of all room numbers that are curr...,SELECT room_number FROM Rooms WHERE status = '...,SELECT DISTINCT Rooms.room_number\nFROM Rooms\...,False,Not Equivalent
